# Import thư viện

In [1]:
import requests
from bs4 import BeautifulSoup
from tqdm.auto import tqdm
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from concurrent.futures import ThreadPoolExecutor
import re
import time 

d:\TLCN\Source Code\GPT test\gptenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Url seed

In [2]:
url_seed = "https://ivie.vn/cong-dong"
url_page_template = "https://ivie.vn/cong-dong?page={}"
url_template = "https://ivie.vn{}"

# Crawling

Chạy song song với 10 workers

In [3]:
data_dict = {
    "specialty": [], 
    "title": [],
    "gender": [],
    "age": [],
    "text": []
}

In [4]:
def get_html(url):
    while True:
        # Gửi yêu cầu HTTP để lấy nội dung HTML
        response = requests.get(url)
        
        if response.status_code == 200:
            html_content = response.text
            return html_content
        
        
        
        print(f"Unable to connect to {url}! Try again in 3 seconds")
        time.sleep(3)
        
def get_html_selenium(url):
    while True:
        try:
            # Khởi động trình duyệt Chrome với WebDriverManager
            driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
            driver.set_page_load_timeout(10)
            # Điều hướng đến trang web
            driver.get(url)

            # Lấy mã HTML của trang
            html_source = driver.page_source
            driver.quit()
            return html_source
        except:
            print(f"Can't get connection to {url}! Try again in 3 seconds.")
            time.sleep(3)

def get_qa(url):
    try:
        html_content = get_html(url)
        soup = BeautifulSoup(html_content, 'html.parser')
        title = soup.find_all('h1')[0].text
        
        QA_detail = []
        elements = soup.select('div.col.d-flex')
        for element in elements:
            QA_detail.append(element.text)
            
        text = ""
        elements = soup.select('.description')
        answered = True
        for i, element in enumerate(elements):
            t = element.text
            match = re.search(r'(Nữ|Nam), (\d+) tuổi(.*)', QA_detail[i], re.DOTALL) # Nếu match thì đây là câu hỏi
            if match and t in QA_detail[i]:
                answered = False
                gender = match.group(1)
                age = match.group(2)
                if text != "":
                    text += "\n"
                else:
                    text += "Người dùng: " + title + " " + t
                    continue
                text += "Người dùng: " + t
            else:
                if text != "":
                    text += "\n"
                if answered == False:
                    text += "Bác sĩ AI:"
                    answered = True
                text += " " + t
    
        specialty = ""
        for e in soup.select('.specialist'):
            specialty += e.text + ", "
        specialty = specialty[:-2]
        return text, specialty, title, gender, age
    except:
        print("Error with this QA!")
        return None

def crawling_one_page(page_id):
    url = url_page_template.format(page_id)
    html_content = get_html_selenium(url)
        
    # Lấy các QA từ các href hợp lệ 
    url_qa = []
    soup = BeautifulSoup(html_content, 'html.parser')
    links = soup.find_all('a')
    for link in links:
        href = link.get('href')
        if href is not None:
            if "/cong-dong?id=" in href:
                url_qa.append(url_template.format(href))
    for u in tqdm(url_qa, desc=f"Processing page {page_id}: "):
        result_qa = get_qa(u)
        if result_qa is not None:
            text, specialty, title, gender, age = result_qa
            data_dict["text"].append(text)
            data_dict["gender"].append(gender)
            data_dict["age"].append(age)
            data_dict["specialty"].append(specialty)
            data_dict["title"].append(title) 

In [5]:
with ThreadPoolExecutor(max_workers=5) as executor:
    results = list(executor.map(crawling_one_page, [i for i in range(1, 4495)]))

Can't get connection to https://ivie.vn/cong-dong?page=3! Try again in 3 seconds.
Can't get connection to https://ivie.vn/cong-dong?page=5! Try again in 3 seconds.
Can't get connection to https://ivie.vn/cong-dong?page=4! Try again in 3 seconds.


Processing page 1: 100%|██████████| 7/7 [00:07<00:00,  1.02s/it]


Can't get connection to https://ivie.vn/cong-dong?page=3! Try again in 3 seconds.


Processing page 5:  40%|████      | 2/5 [00:02<00:03,  1.03s/it]

Processing page 5:  60%|██████    | 3/5 [00:03<00:02,  1.01s/it]


Processing page 5:  80%|████████  | 4/5 [00:04<00:01,  1.09s/it]


Processing page 5: 100%|██████████| 5/5 [00:04<00:00,  1.01it/s]




Processing page 6: 100%|██████████| 7/7 [00:06<00:00,  1.06it/s]



Processing page 4: 100%|██████████| 8/8 [00:07<00:00,  1.14it/s]




Processing page 9:   0%|          | 0/4 [00:00<?, ?it/s]

Can't get connection to https://ivie.vn/cong-dong?page=10! Try again in 3 seconds.


...

Processing page 4481: : 0it [00:00, ?it/s]
Processing page 4492: : 0it [00:00, ?it/s]
Processing page 4490: : 0it [00:00, ?it/s]


Can't get connection to https://ivie.vn/cong-dong?page=4491! Try again in 3 seconds.
Can't get connection to https://ivie.vn/cong-dong?page=4493! Try again in 3 seconds.


Processing page 4489: : 0it [00:00, ?it/s]


Can't get connection to https://ivie.vn/cong-dong?page=4494! Try again in 3 seconds.
Can't get connection to https://ivie.vn/cong-dong?page=4493! Try again in 3 seconds.
Can't get connection to https://ivie.vn/cong-dong?page=4491! Try again in 3 seconds.
Can't get connection to https://ivie.vn/cong-dong?page=4494! Try again in 3 seconds.


Processing page 4493: : 0it [00:00, ?it/s]
Processing page 4491: : 0it [00:00, ?it/s]
Processing page 4494: : 0it [00:00, ?it/s]


# Tách specialty cho Da Liễu

In [6]:
da_lieu_dict = {
    "specialty": [],
    "title": [],
    "gender": [],
    "age": [],
    "text": []
}

# Duyệt qua từng mục trong data_dict
for i in range(len(data_dict["specialty"])):
    if "Da Liễu" in data_dict["specialty"][i]:
        da_lieu_dict["specialty"].append(data_dict["specialty"][i])
        da_lieu_dict["title"].append(data_dict["title"][i])
        da_lieu_dict["gender"].append(data_dict["gender"][i])
        da_lieu_dict["age"].append(data_dict["age"][i])
        da_lieu_dict["text"].append(data_dict["text"][i])

# Lưu data

In [7]:
import pandas as pd
df = pd.DataFrame(data_dict)
df.to_csv("ivie_datacrawling.csv", index=False)
print(f"Số lượng dữ liệu trong ivie_datacrawling.csv cào được: {df.shape[0]}")

df = pd.DataFrame(da_lieu_dict)
df.to_csv("dalieu_ivie_datacrawling.csv", index=False)
print(f"Số lượng dữ liệu trong dalieu_ivie_datacrawling.csv cào được: {df.shape[0]}")

Số lượng dữ liệu trong ivie_datacrawling.csv cào được: 4942
Số lượng dữ liệu trong dalieu_ivie_datacrawling.csv cào được: 644
